In [ ]:
%load_ext autoreload
%autoreload 2

# GenReco - The Lyrics-based Genre Recognizer

The primary objective of this project is to investigate the feasibility of predicting a song's genre based on its lyrical features. With the vast amount of music available today, categorizing songs into genres manually can be time-consuming and subjective.

By leveraging the power of data science techniques, we aim to explore whether the lyrical characteristics of songs can serve as reliable indicators for genre classification. Through this research, we intend to contribute to the field of music analysis and enhance our understanding of the relationship between lyrics and musical genres.

# Procedure

The project will follow a systematic procedure to investigate the predictability of song genres based on lyrical features.

## Study
Initially, extensive research will be conducted to gain a comprehensive understanding of the subject matter, exploring existing studies and theories related to music genre classification.

## Dataset Generation
To generate a suitable dataset for analysis, we will utilize the Spotify API to retrieve relevant song metadata such as artist, album, and genre information. Additionally, we will crawl lyrics from Genius.com, a popular lyrics website, to obtain the lyrical features necessary for our analysis. By combining these sources, we aim to create a diverse and representative dataset that encompasses a wide range of genres and artists.

## Pre-EDA
Once the dataset is obtained, we will focus on understanding its composition and characteristics.
 The characteristics we will be focusing on are the data types and their distributions.
 Exploratory data analysis (EDA) techniques, including visualizations and statistical summaries, will be employed to gain insights into the distribution and relationships between lyrical features and song genres. This exploratory phase will allow us to draw preliminary conclusions and identify any initial patterns or trends within the data, as well as measures of Central Tendency and Variability.

## Data Curation
Next, data curation steps will be applied to ensure the quality and reliability of the dataset. This process will involve the identification and removal of duplicate entries, as well as the detection and handling of outliers or inconsistencies in the data. By carefully curating the dataset, we aim to enhance the accuracy and integrity of our subsequent analysis.

## EDA
Following data curation, an in-depth exploratory data analysis will be conducted. This phase will involve examining the distributions, correlations, and other relevant statistical properties of the lyrical features across different genres. Visualizations such as histograms, scatter plots, and box plots will be employed to facilitate a comprehensive understanding of the dataset and unveil potential insights regarding the relationship between lyrical features and song genres.

## Machine Learning
With a solid grasp of the dataset and its characteristics, we will proceed to the model selection stage. Various machine learning algorithms, such as decision trees, random forests, or KNN, will be evaluated and compared to identify the most suitable model for genre prediction based on lyrical features. Model performance metrics, including accuracy, precision, and recall, will be assessed to determine the effectiveness of each algorithm in capturing the underlying patterns within the data.

## Conclusions
Finally, based on the chosen model and its performance, we will draw final conclusions regarding the project's main objective: the predictability of song genres using lyrical features. The findings of this research endeavor will shed light on the relationship between lyrics and musical genres, providing valuable insights for music analysis and potentially influencing future advancements in genre classification methodologies.

# Some Imports...
Here we are importing the 3rd-party libraries required for the notebook and demonstrations purposes,
as well as some proprietary functionality that we've exported to modules, in order to avoid clutter in the notebook.

In [ ]:
# IMPORT 3RD PARTY LIBRARIES

import os.path as osp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

In [ ]:
# IMPORT UTILS

from utils.ml_utils import split_to_train_and_test, get_classifier_obj, calc_evaluation_val
from utils.curation_utils import transfer_to_categorical, remove_duplicates_and_drop_na, repair_numeric_missing_vals, \
    outlier_detection_iqr, my_dist_to_avg
from utils.plot_utils import plot_frequent_elements, plot_cross_tabulation, get_highly_correlated_cols, \
    transfer_str_to_numeric_vals, plot_frequencies, plot_continuous_feature_relations, plot_histograms
from utils.general_utls import load_dataset
from utils.lyrics_utils import reget_lyrics_df

# Dataset Generation

The generation of the dataset was carried out separately using a regular Python script execution. This approach was chosen to maintain a clear distinction between the dataset and its analysis within this notebook. By treating the dataset as pre-existing data, it allows for better organization of our thought process and prevents mixing the data generation process with its subsequent analysis.

>Having said that, during the process of the data analysis, we will not shy away from manipulating the data to fit our needs.

<i>Refer to generate_dataset.py</i>

# Loading the Dataset

In [ ]:
file_name = "huge_dataset.csv"
dataset_file_path = osp.join(osp.dirname(osp.abspath("__file__")), file_name)
dataset = load_dataset(dataset_file_path)

# Pre-EDA: Data Visualization

Let us examine the individual columns in the data.
The data contains the following fields:

| Column Name      | Data Type   | Variable type |
|------------------|-------------|----|
| source_genre     | object      | Free text |
| name             | object      | Free text |
| artists          | object      | Free text |
| release_year     | int64       | Ordinal |
| release_month    | float64     | Ordinal |
| genres           | object      | List |
| genre     | object      | Categorial |
| duration         | int64       | Numeric|
| popularity       | int64       | Ordinal|
| lyrics_file      | object      | Free text (metadata) |
| lyrics_url       | object      | Free text (metadata) |
| intro_cnt        | float64     | Ordinal|
| outro_cnt        | float64     | Ordinal|
| verse_cnt        | float64     | Ordinal|
| chorus_cnt       | float64     | Ordinal|
| line_cnt         | float64     | Numeric|
| word_cnt         | float64     | Numeric|
| unique_words_cnt | float64     | Ordinal |
| stop_words_cnt   | float64     | Ordinal |
| slang_words_cnt  | float64     | Ordinal |
| positive         | float64     | Ordinal |
| negative         | float64     | Ordinal |
| neutral          | float64     | Ordinal |
| compound         | float64     | Ordinal |

## Top Elements
With Pie Charts, we can see the ratio of the leading elements of each feature

In [ ]:
pie_cols = ['genre', 'artists', 'release_year']
plot_frequencies(dataset, pie_cols, 'pie', 7)

As the graphs show, we can learn that, out of the leading values:
- there are significantly more pop songs than other genres
- the artists are mostly balanced
- big majority of the data is recently released songs

## Ordinal Variables Frequency
With Bar Charts, it's easier to understand how frequent every ordinal variable is, in relation to itself

In [ ]:
bars_cols = ['release_year', 'release_month',
             'intro_cnt', 'outro_cnt', 'verse_cnt', 'chorus_cnt']
plot_frequencies(dataset, bars_cols, 'bar')

As the graphs show, we can learn that:
- as seen before, most of the data is indeed new songs. In addition, the data decreases as the years go back
- suspiciously, there are 0 chorus and verse count, which might indicate an issue with the data
- songs are release relatively on the same rate across the year's month, with a slight increase at the beginning and middle of the year

##

In [ ]:
histogram_cols = ['popularity', 'unique_word_cnt', 'slang_word_cnt',
                  'positive', 'negative', 'neutral', 'compound', 'stop_word_cnt']
plot_histograms(dataset, histogram_cols)

# Conclusions - 1D visualizations

From looking at the graphs we can identify the following:
- Pop is the most popular genre
- Many songs have 0 popularity - but the next "spike" of data is around 80
-

# FEATURE RELATIONS VISUALIZATION

In [ ]:
continuous_vars = ["duration", "line_cnt", "word_cnt", "unique_word_cnt", "stop_word_cnt", "slang_word_cnt"]
plot_continuous_feature_relations(dataset, continuous_vars)

# MISSING VALUES & OUTLIERS HUNT
As part of the data manipulation process, let's test the data's missing values, and look for outlier values

In [ ]:
missing_values = (dataset.isna() | dataset.isnull()).sum() / len(dataset)
missing_values = missing_values[missing_values>0]
plt.figure(figsize=(18, 6))
plt.xticks(rotation=45)
plt.title("Missing values per Column (ratio, non-zero)")
plt.bar(missing_values.index, missing_values.values, width=0.2)

## Analysis: missing values
Some records don't have the release_month field set. This is because the original released date in the dataset included the release year only. Since this is a not-so-significant amount of data, we will get rid of these lines

In [ ]:
dataset_1 = dataset.dropna(axis="index", subset=["release_month"])
len(dataset_1)/len(dataset)

## CLEAN DATASET

In [ ]:
dup_na_removed = remove_duplicates_and_drop_na(dataset)
outliers = outlier_detection_iqr(dup_na_removed, my_dist_to_avg)
repaired = repair_numeric_missing_vals(outliers, dup_na_removed.select_dtypes('number'))
repaired.info()

In [ ]:
plot_frequencies(repaired)

In [ ]:
plot_continuous_feature_relations(repaired)

In [ ]:
numeric_to_bin_dict = {
    "duration": [0, 150000, 300000, 450000, 600000, 750000],
    "popularity": [0, 20, 40, 60, 80, 100],
    "line_cnt": [0, 50, 100, 150, 200, 250],
    "word_cnt": [0, 300, 600, 900, 1200, 1500],
    "unique_words_ratio": [0, 100, 200, 300, 400, 500],
    "stop_words_ratio": [-1, 150, 300, 450, 600, 750],
}

categorical_cols = ['chorus_cnt', 'verse_cnt']
transferred = transfer_to_categorical(repaired, numeric_to_bin_dict, categorical_cols)

In [ ]:
transferred.info()

EDA SECTION

In [ ]:
df_params = pd.DataFrame({'plot_type': ['bar', 'line', 'pie'],
                          'col_name': ['common_genre', 'release_year', 'artists'],
                          'num_top_elements': [6,6,6]})
plot_frequent_elements(transferred, df_params)

In [ ]:
plt.rcParams["figure.figsize"] = (18,6)

In [ ]:
print("\nCommon Genre Distribution:")
sns.countplot(x='common_genre', data=transferred)
plt.show()

In [ ]:
numerical_cols = ['duration','popularity', 'line_cnt', 'word_cnt', 'unique_words_ratio',
                  'stop_words_ratio']
for col in numerical_cols:
    sns.histplot(transferred[col])
    plt.title(col + " Distribution")
    plt.show()

In [ ]:
sentiment_cols = ['negative', 'neutral', 'positive', 'compound']
for col in sentiment_cols:
    sns.histplot(transferred[col])
    plt.title(col + " Distribution")
    plt.show()

In [ ]:
# cross_tabulation(transferred, 'release_date', 'popularity')
plot_cross_tabulation(transferred, ['release_month', 'popularity'], 'common_genre')

In [ ]:
correlations, tuple_arr = get_highly_correlated_cols(transferred)

In [ ]:
cols_for_correlations = transferred.columns
indx_sort = np.argsort(correlations)
for n_correlation in indx_sort:
    col_lt, col_rt = tuple_arr[n_correlation]
    col_name_lt, col_name_rt = cols_for_correlations[col_lt], cols_for_correlations[col_rt]
    title = "corr('%s', '%s')=%4.2f" %(col_name_lt, col_name_rt, correlations[n_correlation])
    print(title)

In [ ]:
transferred = transfer_str_to_numeric_vals(transferred)
transferred.info()

In [ ]:
prepared = transferred.drop(
    columns=[
        'genres', 'source_genre',
        'name', 'lyrics_file',
        'lyrics_url', 'popularity',
        'compound', 'neutral',
        'line_cnt'
    ],
    axis=1
)

In [ ]:
prepared.info()

In [ ]:
X_train, X_test, y_train, y_test = split_to_train_and_test(prepared, 'common_genre', 0.2, 5)

params = {'n_neighbors':23}
clf = get_classifier_obj("KNN",params)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
accuracy_val = calc_evaluation_val("accuracy", y_test, y_predicted)
print(accuracy_val)
# matrix = calc_evaluation_val("confusion_matrix", y_test, y_predicted)
# print(matrix)
# recall_val = calc_evaluation_val("recall", y_test, y_predicted)
# print(recall_val)
# f1_val = calc_evaluation_val("f1", y_test, y_predicted)
# print(f1_val)
# confusion_matrix_val = calc_evaluation_val("confusion_matrix", y_test, y_predicted)